# **Day 10: Pipe Maze**

# Setup
The cells below will set up the rest of the notebook. 

I'll start by configuring my kernel:

In [1]:
# Changing the current working directory
%cd ..

# Enabling the autoreload extension
%load_ext autoreload
%autoreload 2

d:\data\programming\advent-of-code-2023


Now, I'm going to import some libraries:

In [2]:
# Import statements
import pandas as pd
import re
from tqdm import tqdm

Finally, I'll load in the data for this puzzle. 

In [3]:
# Load in the data for the puzzle
day = 10
input_data_path = f"data/input-files/day-{day:02d}-input.txt"
example_data_path = f"data/example-input/day-{day:02d}-example.txt"
with open(input_data_path, "r") as txt_file:
    input_data = txt_file.readlines()

# Parsing a Network from the Maze 
I need to parse the maze using the following rules: 

```
| is a vertical pipe connecting north and south.
- is a horizontal pipe connecting east and west.
L is a 90-degree bend connecting north and east.
J is a 90-degree bend connecting north and west.
7 is a 90-degree bend connecting south and west.
F is a 90-degree bend connecting south and east.
. is ground; there is no pipe in this tile.
S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.
```

I'm going to make a network-like structure. 

In [4]:
# First, parse the input_data into a coordinate grid
maze_coordinate_grid = [[x for x in line.strip()] for line in input_data]

# Set up a dictionary of edges
edges = {}

# Indicate the possible directions travelable from each pipe
possible_directions_from_pipes = {
    "|": [(0, 1), (0, -1)],
    "-": [(-1, 0), (1, 0)],
    "L": [(0, 1), (1, 0)],
    "J": [(0, 1), (-1, 0)],
    "7": [(-1, 0), (0, -1)],
    "F": [(1, 0), (0, -1)],
}

# Iterate through each cell in the maze and determine the possible moves from there
for row_idx, row in enumerate(maze_coordinate_grid):
    for col_idx, pipe_type in enumerate(row):
        # If the pipe_type isn't in the possible_directions_from_pipes keys, skip it
        if pipe_type not in possible_directions_from_pipes:
            continue

        # Iterate through each of the possible nodes connected to this one
        for possible_directions in possible_directions_from_pipes.get(pipe_type):
            cur_coords = (col_idx, row_idx)
            next_coords = (
                col_idx + possible_directions[0],
                row_idx - possible_directions[1],
            )
            
            # Add this connection to the edges
            if cur_coords not in edges:
                edges[cur_coords] = []
            edges[cur_coords].append(next_coords)

Now, with this structure in hand, we should be able to write a method to iterate through different paths till S is found. 

In [5]:
def traverse_maze_from_position(starting_pos, edges, maze, reverse_traversal=False):
    """
    This method will traverse the maze from a particular starting position. 
    """
    
    # We're going to keep a path 
    cur_path = [starting_pos]
    cur_pos = starting_pos
    cur_node = maze[cur_pos[1]][cur_pos[0]]
    while cur_node != "S":
        possible_new_nodes = edges.get(cur_pos)
        
        if reverse_traversal == True:
            possible_new_nodes = reversed(possible_new_nodes)
        
        for possible_new_node in possible_new_nodes:
            if possible_new_node not in cur_path:
                cur_path.append(possible_new_node)
                cur_pos = possible_new_node
                cur_node = maze[cur_pos[1]][cur_pos[0]]
                break
    
    # Return the traversal path 
    return cur_path

In [6]:
# Now: we're going to identify each node around the S, and then determine the longest paths
pipes_around_animal_start = []
for row_idx, row in enumerate(maze_coordinate_grid):
    for col_idx, pipe_type in enumerate(row):
        if pipe_type == "S":
            for transformation in [
                (0, 1),
                (1, 0),
                (0, -1),
                (-1, 0),
            ]:
                # Determine the new coordinate 
                new_coord = (col_idx + transformation[0], row_idx - transformation[1])
                if new_coord[0] < 0 or new_coord[0] >= len(maze_coordinate_grid[0]):
                    continue
                elif new_coord[1] < 0 or new_coord[1] >= len(maze_coordinate_grid):
                    continue
                
                # Determine the pipe type at the new coordinate
                pipe_at_new_coord = maze_coordinate_grid[new_coord[1]][new_coord[0]]
                if pipe_at_new_coord != "." and pipe_at_new_coord != "S":
                    pipes_around_animal_start.append(new_coord)

In [7]:
longest_path_from_starting_pipe_coords = {}
for starting_pipe_coords in pipes_around_animal_start:
    # Determine the longer path 
    longest_path = traverse_maze_from_position(
        starting_pos=starting_pipe_coords,
        edges=edges,
        maze=maze_coordinate_grid,
        reverse_traversal=True
    )

    other_path = traverse_maze_from_position(
        starting_pos=starting_pipe_coords,
        edges=edges,
        maze=maze_coordinate_grid,
        reverse_traversal=False
    )

    if len(other_path) > len(longest_path):
        longest_path = other_path
    
    longest_path_from_starting_pipe_coords[starting_pipe_coords] = longest_path

In [14]:
int(max([len(longest_path) for starting_coord, longest_path in longest_path_from_starting_pipe_coords.items()])/2)

6979